In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
import os

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor

# local import
from make_datasets import make_data
import models
import evaluation

2023-08-22 00:13:47.787411: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = '/Users/jyontika/Desktop/jyontika-MA-data/data'
data_path= os.path.join(data_dir, './clean_annual_tract/')
data_gdf = gpd.read_file(data_path)

Process dataframe into a data frame with a Multiindex on location and time

In [3]:

# Name the important columns
timestep_col = 'timestep'
geography_col = 'geoid'
outcome_col = 'deaths'

# These are the columns we could possibly want in the X dataframe
x_idx_cols = [geography_col, 'lat', 'lon', timestep_col,
              'theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc',
              'svi_pctile', 'year',
              'neighbor_t', 'deaths']

# These are the columns we could want in the Y dataframe
y_idx_cols = [geography_col, timestep_col, outcome_col]

# These are the features we want
features_only = ['deaths']
add_spacetime = True
add_svi = True
if add_spacetime:
    features_only += ['lat', 'lon', timestep_col]
if add_svi:
    features_only += ['theme_1_pc', 'theme_2_pc', 'theme_3_pc', 'theme_4_pc', 'svi_pctile']


validation_year = 2018
first_test_year = 2019
last_test_year = 2020
first_test_timestep = 19
last_test_timestep = 20
lookback_years= 5
first_train_eval_year = validation_year - lookback_years
last_train_eval_year = validation_year -1

In [4]:
# Create the multiindex
multiindexed_gdf = data_gdf.set_index([geography_col, timestep_col])

# re-add the timestep column as a feature because it's useful
multiindexed_gdf[timestep_col] = multiindexed_gdf.index.get_level_values(timestep_col)

# Track number of locations
num_geoids = len(data_gdf[geography_col].unique())

In [5]:
x_BSF, y_BS = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          features_only, num_geoids)
x_test_BSF, y_test_BS =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          features_only, num_geoids)

# For the weighted historical average model, we only use deaths as features
x_BSF_death_only, y_BS_death_only = make_data(multiindexed_gdf, first_train_eval_year, last_train_eval_year, lookback_years,
          ['deaths'], num_geoids)
x_test_BSF_death_only, y_test_BS_death_only =make_data(multiindexed_gdf, first_test_year, last_test_year, lookback_years,
          ['deaths'], num_geoids)

In [6]:
x_BSF.shape

TensorShape([5, 1620, 45])

In [7]:
y_BS.shape

TensorShape([5, 1620])

In [8]:
all_evaluation_deaths = []

for timestep in range(first_test_timestep, last_test_timestep+1):
    evaluation_deaths = multiindexed_gdf.loc[idx[:, timestep], :]
    evaluation_deaths = evaluation_deaths.drop(columns=timestep_col).reset_index().set_index('geoid')['deaths']
    
    # Append the current evaluation_deaths data to the list
    all_evaluation_deaths.append(evaluation_deaths)

evaluation_deaths = all_evaluation_deaths

### All Zeroes Model
#### lookback years =2

In [9]:
bpr_over_time_zeroes, predicted_over_time_zeroes = models.all_zeroes_model(multiindexed_gdf,
                                        first_test_timestep, last_test_timestep,
                                        num_geoids, bpr_uncertainty_samples=15)

In [10]:
print(f"2019 Average: {np.mean(bpr_over_time_zeroes[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_zeroes[0]) + \
                          np.array(bpr_over_time_zeroes[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.25896326804183245
Zeroes model (Mean, 95% CI): 25.4,
      (25.1-
       25.8)


In [11]:
zeroes_rmse_results, zeroes_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_zeroes)

In [12]:
zeroes_rmse_mean, zeroes_rmse_conf_interval = zeroes_rmse_results
zeroes_mae_mean, zeroes_mae_conf_interval = zeroes_mae_results

evaluation.print_results("RMSE for Zeroes Model", zeroes_rmse_mean, zeroes_rmse_conf_interval)
evaluation.print_results("MAE for Zeroes Model", zeroes_mae_mean, zeroes_mae_conf_interval)


RMSE for Zeroes Model (Mean, 95% CI): 1.80, (1.73-1.88)
MAE for Zeroes Model (Mean, 95% CI): 1.17, (1.12-1.22)


### Last Year
#### lookback = 1 year

In [13]:
bpr_over_time_last_time, predicted_over_time_last_time  = models.last_time_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1, bpr_uncertainty_samples=50)

In [14]:
print(f"2019 Average: {np.mean(bpr_over_time_last_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_last_time[0]) + \
                          np.array(bpr_over_time_last_time[1]))/2
                        
print(f"""Zeroes model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5583644190051337
Zeroes model (Mean, 95% CI): 53.9,
      (52.4-
       55.3)


In [15]:
len(predicted_over_time_last_time)

2

In [16]:
last_time_rmse_results, last_time_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_last_time)

In [17]:
last_time_rmse_mean, last_time_rmse_conf_interval = last_time_rmse_results
last_time_mae_mean, last_time_mae_conf_interval = last_time_mae_results

evaluation.print_results("RMSE for Last Year Model", last_time_rmse_mean, last_time_rmse_conf_interval)
evaluation.print_results("MAE for Last Year Model", last_time_mae_mean, last_time_mae_conf_interval)


RMSE for Last Year Model (Mean, 95% CI): 1.51, (1.48-1.54)
MAE for Last Year Model (Mean, 95% CI): 1.05, (1.03-1.08)


### Historical Average 
#### lookback = 1 years

In [18]:
bpr_over_time_avg_time, predicted_over_time_avg_time = models.historical_average_model(multiindexed_gdf, first_test_timestep, last_test_timestep, num_geoids,
                     1, 7, bpr_uncertainty_samples=10)

In [19]:
print(f"2019 Average: {np.mean(bpr_over_time_avg_time[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_avg_time[0]) + \
                          np.array(bpr_over_time_avg_time[1]))/2
                        
print(f"""Hist. Avg  model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6308297301973017
Hist. Avg  model (Mean, 95% CI): 61.0,
      (59.9-
       62.5)


In [20]:
avg_time_rmse_results, avg_time_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_avg_time)

In [21]:
avg_time_rmse_mean, avg_time_rmse_conf_interval = avg_time_rmse_results
avg_time_mae_mean, avg_time_mae_conf_interval = avg_time_mae_results

evaluation.print_results("RMSE for Historical Average Model", avg_time_rmse_mean, avg_time_rmse_conf_interval)
evaluation.print_results("MAE for Historical Average Model", avg_time_mae_mean, avg_time_mae_conf_interval)


RMSE for Historical Average Model (Mean, 95% CI): 1.20, (1.16-1.25)
MAE for Historical Average Model (Mean, 95% CI): 0.87, (0.84-0.89)


### Weighted Historical Average
#### lookback = 7 years

Make Scikit models

In [22]:
# Identical models, features are only difference
linear_poisson_weighted_avg = sklearn.linear_model.PoissonRegressor()
linear_poisson = sklearn.linear_model.PoissonRegressor()

# Params selected via grid search on validation. Need to re-do grid search for chicago
hist_poisson =   HistGradientBoostingRegressor(loss="poisson", max_iter=10000, max_depth=3, max_leaf_nodes=2,
                                               l2_regularization=1, min_samples_leaf=100 )

In [23]:
bpr_over_time_weight_avg, predicted_over_time_weight_avg = models.scikit_model(multiindexed_gdf, x_BSF_death_only,
                                               y_BS_death_only, x_test_BSF_death_only,
                                               linear_poisson_weighted_avg,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [ ]:
print(f"2019 Average: {np.mean(bpr_over_time_weight_avg[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_weight_avg[0]) + \
                          np.array(bpr_over_time_weight_avg[1]))/2
                        
print(f"""Weighted Hist. Avg  model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.62648403705886
Weighted Hist. Avg  model (Mean, 95% CI): 61.1,
      (59.2-
       63.4)


In [ ]:
weight_avg_rmse_results, weight_avg_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_weight_avg)

In [ ]:
weight_avg_rmse_mean, weight_avg_rmse_conf_interval = weight_avg_rmse_results
weight_avg_mae_mean, weight_avg_mae_conf_interval = weight_avg_mae_results

evaluation.print_results("RMSE for Weighted Average Model", weight_avg_rmse_mean, weight_avg_rmse_conf_interval)
evaluation.print_results("MAE for Weighted Average Model", weight_avg_mae_mean, weight_avg_mae_conf_interval)


RMSE for Weighted Average Model (Mean, 95% CI): 1.38, (1.16-1.62)
MAE for Weighted Average Model (Mean, 95% CI): 1.03, (0.89-1.19)


In [ ]:
# df_weighted = models.scikit_model_with_coefficients(multiindexed_gdf, x_BSF_death_only,
#                                                y_BS_death_only, x_test_BSF_death_only,
#                                                linear_poisson_weighted_avg,
#                                                first_test_timestep, last_test_timestep,
#                                                bpr_uncertainty_samples=20)


[0.07027757 0.06632171 0.07620279 0.12114664 0.13987368]


In [ ]:
# #df_weighted
# excel_filename = 'weighted_average_df_MA.xlsx'  # Provide the desired filename
# df_weighted.to_excel(excel_filename, index=False)  # Specify index=False to exclude row indices from the output


### Linear (Poisson GLM baseline)

In [ ]:
bpr_over_time_linear, predicted_over_time_linear = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               linear_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=50)

/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


In [ ]:
print(f"2019 Average: {np.mean(bpr_over_time_linear[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_linear[0]) + \
                          np.array(bpr_over_time_linear[1]))/2
                        
print(f"""Linear (Poisson GLM) model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.6320362020652359
Linear (Poisson GLM) model (Mean, 95% CI): 61.5,
      (59.8-
       63.6)


In [ ]:
linear_rmse_results, linear_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_linear)

In [ ]:
linear_rmse_mean, linear_rmse_conf_interval = linear_rmse_results
linear_mae_mean, linear_mae_conf_interval = linear_mae_results

evaluation.print_results("RMSE for Linear (Poisson GLM)", linear_rmse_mean, linear_rmse_conf_interval)
evaluation.print_results("MAE for Linear (Poisson GLM)", linear_mae_mean, linear_mae_conf_interval)


RMSE for Linear (Poisson GLM) (Mean, 95% CI): 1.49, (1.23-1.76)
MAE for Linear (Poisson GLM) (Mean, 95% CI): 1.15, (0.98-1.34)


In [ ]:
# df_linear = models.scikit_model_with_coefficients(multiindexed_gdf, x_BSF,
#                                                y_BS, x_test_BSF,
#                                                linear_poisson,
#                                                first_test_timestep, last_test_timestep,
#                                                bpr_uncertainty_samples=20)

[ 6.18966478e-02 -8.25036641e-05  1.67316266e-02  1.74183118e-02
  3.46454029e-02  2.71918272e-02  1.38496681e-02  1.49714283e-02
  2.99307343e-02  5.82961586e-02 -8.25036641e-05  1.67316266e-02
  1.71992294e-02  3.51952671e-02  2.90108855e-02  1.34023123e-02
  1.47802892e-02  3.03328484e-02  5.87837416e-02 -8.25036641e-05
  1.67316266e-02  1.69801471e-02  3.51724081e-02  3.00596110e-02
  1.32098422e-02  1.48401518e-02  3.05241746e-02  9.51523868e-02
 -8.25036641e-05  1.67316266e-02  1.67610648e-02  3.48930053e-02
  2.97506359e-02  1.32414517e-02  1.38237260e-02  3.02800897e-02
  1.13193152e-01 -8.25036641e-05  1.67316266e-02  1.65419824e-02
  3.41956241e-02  2.90526656e-02  1.34666813e-02  1.44635428e-02
  3.02394886e-02]


/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/Users/jyontika/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_glm/glm.py:282: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Inc

KeyboardInterrupt: 

### Gradient Boosted Trees (Poisson)

In [ ]:
bpr_over_time_tree, predicted_over_time_tree = models.scikit_model(multiindexed_gdf, x_BSF,
                                               y_BS, x_test_BSF,
                                               hist_poisson,
                                               first_test_timestep, last_test_timestep,
                                               bpr_uncertainty_samples=20)

In [ ]:
print(f"2019 Average: {np.mean(bpr_over_time_tree[0])}")

bpr_samples_both_years = (np.array(bpr_over_time_tree[0]) + \
                          np.array(bpr_over_time_tree[1]))/2
                        
print(f"""Gradient Boosted Trees (Poisson)  (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")


2019 Average: 0.5583002425125425
Gradient Boosted Trees (Poisson)  (Mean, 95% CI): 57.6,
      (56.3-
       59.3)


In [ ]:
tree_rmse_results, tree_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_over_time_tree)

In [ ]:
tree_rmse_mean, tree_rmse_conf_interval = tree_rmse_results
tree_mae_mean, tree_mae_conf_interval = tree_mae_results

evaluation.print_results("RMSE for Gradient Boosted Trees", tree_rmse_mean, tree_rmse_conf_interval)
evaluation.print_results("MAE for Gradient Boosted Trees", tree_mae_mean, tree_mae_conf_interval)


RMSE for Gradient Boosted Trees (Mean, 95% CI): 1.47, (1.39-1.55)
MAE for Gradient Boosted Trees (Mean, 95% CI): 1.14, (1.05-1.24)


### CASTNet

In [ ]:
# Call the castnet_model function to calculate BPR for CASTNet predictions
bpr_results_castnet, predicted_results_castnet = models.castnet_model(multiindexed_gdf, False, first_test_timestep, last_test_timestep, 
                                            num_geoids, bpr_uncertainty_samples=50)


In [ ]:
print(f"2019 Average: {np.mean(bpr_results_castnet[0])}")

bpr_samples_both_years = (np.array(bpr_results_castnet[0]) + \
                          np.array(bpr_results_castnet[1]))/2
                        
print(f"""CASTNet model (Mean, 95% CI): {np.mean(bpr_samples_both_years)*100:.1f},
      ({np.percentile(bpr_samples_both_years,2.5)*100:.1f}-
       {np.percentile(bpr_samples_both_years,97.5)*100:.1f})""")

2019 Average: 0.2796877101780102
CASTNet model (Mean, 95% CI): 29.0,
      (27.8-
       30.4)


In [ ]:
castnet_rmse_results, castnet_mae_results  = evaluation.calculate_metrics(evaluation_deaths, predicted_results_castnet)

In [ ]:
castnet_rmse_mean, castnet_rmse_conf_interval = castnet_rmse_results
castnet_mae_mean, castnet_mae_conf_interval = castnet_mae_results

evaluation.print_results("RMSE for CASTNet", castnet_rmse_mean, castnet_rmse_conf_interval)
evaluation.print_results("MAE for CASTNet", castnet_mae_mean, castnet_mae_conf_interval)


RMSE for CASTNet (Mean, 95% CI): 1.91, (1.73-2.10)
MAE for CASTNet (Mean, 95% CI): 1.53, (1.33-1.73)
